# 라이브러리

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
import getpass
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4-turbo")


## if source == 지연:


In [4]:
def text_generation(source,prompt_template,category="delay"):
    
    prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(
    """Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    """
    ),
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )
    conversation({"question":"subway delay information(article) :"+source_content+ "Just REVIEW subway delay information"})
    if category=="delay":# 지연
        for i in ["지연/사고 일시","지연/사고 노선","지연/사고 이유"]:
            r=conversation({"question":"Using the provided information \n write "+i+":"})
    elif category == "strike":#파업
        pass
    else: #시간표 변경
        pass
    conversation({"question":prompt_template})
    conversation({"question":"1. 뉴스기사의 내용을 학습해 2. 학습한 뉴스기사와 블로그글을 비교해 3.블로그 글에 틀린 정보가 있다면 수정해 뉴스기사: "+source_content+"블로그 글 :"})
    conversation({"question":"1. 블로그 글의 특징에 맞는 글인지 확인해 2. 블로그 글의 특징에 맞지 않다면 블로그 글처럼 수정해줘"})
    result=conversation({"question":prompt_template})
    return result["text"]

## 지연/사고 템플릿

In [2]:
prompt_template ="""
너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.
아래의 형식으로 작성해줘:

{input}

1) 제목과 본문으로 구분해서 출력해줘.
2) 제목은 창의력 있고, 주목도 있게 구성해줘.
3) 본문을 구성할 때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘
4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘
5) 뉴스기사를 이용해 블로그로 작성한 글과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글 처럼 통통튀고 발랄한 말투로 작성해줘
6) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요

템플릿:
{{여기에 블로그 제목}}
{{시작하는 말}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

{{지연/사고 일시}}
{{지연/사고 노선}}
{{지연/사고 이유}}
{{문의 사항 링크}}

{{뉴스기사를 이용해 블로그로 작성한 글}}

{{마무리 말}}
오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. 
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349
"""


## 실험

## 기사 텍스트로 주입

In [3]:
source_link="https://www.wikitree.co.kr/articles/915949"
source_content= """'번쩍'하더니 폭발음...어제 퇴근길 난리 난 지하철 1호선 상황 (+영상)
2024-01-04 10:06 김희은 기자
승객 500여 명 하차한 뒤 환승...일대 혼잡 등 불편 겪어
퇴근길 서울지하철 1호선에서 전동차 고장으로 한바탕 소동이 일었다.

역곡역 /연합뉴스
지난 3일 SBS뉴스 등 보도에 따르면 3일 오후 5시 54분께 경인선(서울지하철 1호선) 전동차가 온수역에서 역곡역으로 운행하던 중 고장으로 멈췄다.

이 사고로 열차에 타고 있던 승객 500여 명이 10여 분 동안 터널에 정차한 채 대기했고, 다음 역인 역곡역에서 하차한 뒤 환승하는 불편을 겪었다.

또 이 열차 고장으로 인해 경인선 하행선(인천 방면) 오류역∼부천역 구간 후속 열차 운행이 10여 분간 지연됐다.

사고 당시 상황이 담긴 영상도 공개됐다. 전동차 유리창 밖으로 번쩍하는 섬광과 함께 폭발음이 들리더니 갑작스럽게 불꽃이 튀기기도 했다. 놀란 승객들은 "어머, 어떡해", “얼른 다 내려야겠다”며 자리에서 일어나 당황스러움을 내비쳤다.

1호선 열차 고장 장면 / SBS뉴스
1호선 열차 고장 장면...폭발음과 함께 불꽃 / SBS뉴스
이와 관련해 코레일 관계자는 "경인선 상행선(용산 방면) 열차 운행은 지연되지 않았다"라며 정확한 사고 원인을 조사 중이라는 입장을 밝혔다.

앞서 지난해 10월엔 서울지하철 6호선 열차가 문을 연 채 운행하는 일이 벌어졌다.

사고는 퇴근 시간대 발생했다. 19일 오후 6시 30분쯤 6호선 응암행 열차가 월곡역에서 출입문 안쪽에 콘크리트 조각이 끼면서 문이 닫히지 않은 상태로 달렸다.

서울교통공사는 고려대역에서 승객들을 모두 내리게 한 뒤 열차를 차량기지로 보내는 과정에서 이물질을 제거하고 오후 6시 56분쯤 삼각지역에서 다시 승객을 태웠다.

그러나 오후 7시 31분 해당 열차는 연신내역에서 다시 문이 닫히지 않은 장애가 빚어져 운행을 전면 중단하고 차량기지로 돌려보내졌다.

다행히 이 과정에서 다친 사람은 없었지만, 혼잡을 빚는 등 승객들이 불편을 겪었다.


"""


## 기사 정보 학습 (Chaining)

In [28]:
prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(
    """Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    """
    ),
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation({"question":"subway delay information(article) :"+source_content+ "Just REVIEW subway delay information"})
for i in ["지연/사고 일시","지연/사고 노선","지연/사고 이유"]:
    r=conversation({"question":"Using the provided information \n write "+i+":"})


/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    
Human: subway delay information(article) :'번쩍'하더니 폭발음...어제 퇴근길 난리 난 지하철 1호선 상황 (+영상)
2024-01-04 10:06 김희은 기자
승객 500여 명 하차한 뒤 환승...일대 혼잡 등 불편 겪어
퇴근길 서울지하철 1호선에서 전동차 고장으로 한바탕 소동이 일었다.

역곡역 /연합뉴스
지난 3일 SBS뉴스 등 보도에 따르면 3일 오후 5시 54분께 경인선(서울지하철 1호선) 전동차가 온수역에서 역곡역으로 운행하던 중 고장으로 멈췄다.

이 사고로 열차에 타고 있던 승객 500여 명이 10여 분 동안 터널에 정차한 채 대기했고, 다음 역인 역곡역에서 하차한 뒤 환승하는 불편을 겪었다.

또 이 열차 고장으로 인해 경인선 하행선(인천 방면) 오류역∼부천역 구간 후속 열차 운행이 10여 분간 지연됐다.

사고 당시 상황이 담긴 영상도 공개됐다. 전동차 유리창 밖으로 번쩍하는 섬광과 함께 폭발음이 들리더니 갑작스럽게 불꽃이 튀기기도 했다. 놀란 승객들은 "어머, 어떡해", “얼른 다 내려야겠다”며 자리에서 일어나 당황스러움을 내비쳤다.

1호선 열차 고장 장면 / SBS뉴스
1호선 열차 고장 장면...폭발음과 함께 불꽃 / SBS뉴스
이와 관련해 코레일 관계자는 "경인선 상행선(용산 방면) 열차 운행은 지연되지 않았다"라며 정확한 사고 원인을 조사 중이라는 입장을 밝혔다.

앞서 지난해 10월엔 서울지하철 6호선 열차가 문을 연 채 운행하는 일이 벌어졌다.

사고는 퇴근 시간대 발생했다. 19일 오후 6

## 글생성
result1: 첫번째 생성  
result2: 기사와 블로그 글 비교후 수정( 정확성 향상 )  
result3:블로그 글 특징에 맞도록  
result4: 탬플릿에 맞도록

## result1

In [29]:
result1=conversation({"question":prompt_template})



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    
System: **Subway Delays Cause Commuter Chaos in Seoul**

On January 3rd, at approximately 5:54 PM, a train malfunction occurred between Onsu Station and Eokgok Station on the Gyeongin Line of the Seoul Metropolitan Subway, affecting over 500 passengers. These travelers were stranded inside a tunnel for about ten minutes before disembarking at Eokgok Station. The incident caused not only immediate distress among the passengers, evidenced by dramatic eyewitness videos but also subsequent delays that impacted other commuters' schedules towards Incheon.

Although northbound trains towards Yongsan were not delayed, the event has prompted Korail to investigate the exact cause of the malfunction. This malfunction follows a previous incident in October on Line 6, whe

In [30]:
print(result1["text"])

{{서울 지하철 지연 사태: 경인선의 뜻밖의 사고에 대해 알아보자}}
{{시작하는 말}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

{{지연/사고 일시}}
2023년 1월 3일 오후 5시 54분
{{지연/사고 노선}}
서울 지하철 1호선, 경인선 (온수역과 어곡역 사이)
{{지연/사고 이유}}
기차 고장
{{문의 사항 링크}}
https://www.korail.com/

{{뉴스기사를 이용해 블로그로 작성한 글}}
지난 1월 3일, 서울의 출퇴근 시간대에 많은 시민들이 이용하는 경인선에서 뜻밖의 사고가 발생했습니다. 온수역과 어곡역 사이에서 기차가 갑자기 멈춰서는 바람에, 승객들이 약 10분간 터널 안에 갇혀 있었습니다. 다행히 큰 부상자는 없었지만, 이 사건은 다른 승객들의 일정에도 큰 차질을 빚었습니다. 이 사건을 계기로 서울의 공공교통 시스템의 낡은 인프라와 유지관리의 중요성이 다시 한번 부각되었습니다.

{{마무리 말}}
오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다. 지하철 파업으로 아침마다 출근하기 힘드시죠? 저희 '지하철 온다'로 빠르고 편리하게 출근하세요!

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349


## result2 

In [31]:
result2=conversation({"question":"1. 뉴스기사의 내용을 학습해 2. 학습한 뉴스기사와 블로그글을 비교해 3.블로그 글에 틀린 정보가 있다면 수정해 뉴스기사: "+source_content+"블로그 글 :"})
print(result2)



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    
System: **Subway Delays Cause Commuter Chaos in Seoul**

On January 3rd, at approximately 5:54 PM, a train malfunction occurred between Onsu Station and Eokgok Station on the Gyeongin Line of the Seoul Metropolitan Subway, affecting over 500 passengers. These travelers were stranded inside a tunnel for about ten minutes before disembarking at Eokgok Station. The incident caused not only immediate distress among the passengers, evidenced by dramatic eyewitness videos but also subsequent delays that impacted other commuters' schedules towards Incheon.

Although northbound trains towards Yongsan were not delayed, the event has prompted Korail to investigate the exact cause of the malfunction. This malfunction follows a previous incident in October on Line 6, whe

In [32]:
print(result2["text"])

블로그 글에 대한 수정 사항을 제안하면서, 원래 뉴스기사의 내용을 바탕으로 정보 수정을 진행하겠습니다.

**블로그글 수정 전:**
1. "경인선 지하철 사고는 온수역과 역곡역 사이에서 발생하였다"라고 언급되어 있으나, 실제 뉴스에서는 온수역에서 역곡역으로 이동 중이었다는 정보가 제공됩니다.
2. 블로그 글에서는 구체적인 사고의 세부 내용(예: 불꽃과 폭발음)에 대한 언급이 부족합니다.
3. 나중에 하차 후 환승에 대한 불편함이 언급되어 있으나, 구체적인 후속 열차 지연 정보가 누락되었습니다.

**수정된 블로그 글:**
"서울 지하철 지연 사태: 경인선의 뜻밖의 사고에 대해 알아보자" (Seoul Subway Delay Crisis: A Closer Look at the Unexpected Incident on the Gyeongin Line)

지난 3일 오후, 서울 지하철 1호선 경인선에서 예기치 못한 사고가 발생했습니다. 오후 5시 54분경, 온수역에서 역곡역으로 향하던 중 전동차가 고장 나면서 승객 500여 명이 열차 내부에서 약 10분간 터널 안에서 대기해야 했습니다. 이 사고로 인해 역곡역에서 하차한 승객들은 환승에 큰 불편을 겪었으며, 후속 열차 운행도 오류역에서 부천역 구간까지 약 10분간 지연되었습니다.

특히 이번 사고는 열차 외부에서 번쩍이는 섬광과 폭발음이 동반되었으며, 이로 인해 많은 승객들이 당황하고 불안을 느꼈습니다. 이러한 상황은 SBS뉴스를 통해 공개된 영상에서도 확인할 수 있습니다.

코레일은 현재 정확한 사고 원인을 조사 중이며, 경인선 상행선(용산 방면)은 사고의 영향을 받지 않아 지연되지 않았다고 밝혔습니다. 이번 사고는 서울의 지하철 시스템의 안전성 및 신뢰성에 대한 중요한 질문을 다시 한번 제기하게 했습니다. 우리는 이러한 사고가 재발하지 않도록 지속적인 유지보수와 시스템 개선이 필요함을 다시 한번 느낍니다.

이 블로그는 앞으로도 서울 지하철 시스템의 상태에 대한 최신 정보를 제공할 것입니다. 자주 방문하여 서울 지

## result3

In [33]:
result3=conversation({"question":"1. 블로그 글의 특징에 맞는 글인지 확인해 2. 블로그 글의 특징에 맞지 않다면 블로그 글처럼 수정해줘"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    
System: **Subway Delays Cause Commuter Chaos in Seoul**

On January 3rd, at approximately 5:54 PM, a train malfunction occurred between Onsu Station and Eokgok Station on the Gyeongin Line of the Seoul Metropolitan Subway, affecting over 500 passengers. These travelers were stranded inside a tunnel for about ten minutes before disembarking at Eokgok Station. The incident caused not only immediate distress among the passengers, evidenced by dramatic eyewitness videos, but also subsequent delays that impacted other commuters' schedules towards Incheon.

Although northbound trains towards Yongsan were not delayed, the event has prompted Korail to investigate the exact cause of the malfunction. This malfunction follows a previous incident in October on Line 6, wh

In [34]:
print(result3["text"])

블로그 글 작성 시 고려해야 할 특징은 다음과 같습니다:

1. **독자 친화적 어조**: 블로그 글은 일반적으로 따뜻하고 친근한 어조를 사용하여 독자와의 친밀감을 형성합니다.
2. **개인적인 관점과 경험의 추가**: 블로그는 종종 작성자의 개인적인 경험과 관점을 포함하여 독자에게 더욱 명확하고 관련성 있는 정보를 제공합니다.
3. **시각적 요소 포함**: 이미지, 비디오 또는 인포그래픽을 포함하여 글의 이해를 도우며 독자의 관심을 유지합니다.
4. **대화형 질문 사용**: 독자의 참여를 유도하기 위해 질문을 던지거나 피드백을 요청하는 방식으로 글을 마무리합니다.

현재 제공된 글은 뉴스 보도의 정확성과 상세함을 강조하며 정보 전달에 집중되어 있습니다. 그러나 블로그 특유의 친근감과 독자와의 상호작용은 다소 부족합니다. 이를 개선하기 위해 다음과 같이 수정할 수 있습니다:

---

**서울 지하철 지연 사태: 경인선의 뜻밖의 사고에 대해 알아보자**

안녕하세요, 서울 지하철을 자주 이용하시는 여러분! 지난 1월 3일 오후 5시 54분경, 우리가 일상적으로 의지하는 경인선에서 뜻밖의 사고가 발생했습니다. 온수역과 억곡역 사이에서 열차 고장이 발생해, 500명이 넘는 승객들이 약 10분 동안 터널 안에 갇혔다가 억곡역에서 하차했죠. 이 사고로 인해 인천 방향으로 가는 많은 승객들의 일정에 차질이 생겼습니다.

### 사고 현장의 긴장감

이번 사고는 승객들 사이에서 큰 불안과 긴장을 야기했습니다. 외부에서 보이는 스파크와 폭발음이 들리면서, 많은 이들이 공포를 느꼈어요. 다행히 큰 부상자는 발생하지 않았지만, 이런 사고들이 우리의 안전과 신뢰성에 대한 우려를 다시 한번 불러일으키고 있습니다.

### 지속적인 관심이 필요한 이유

서울의 지하철 시스템은 날이 갈수록 이용하는 사람이 많아지고 있습니다. 이런 상황에서 안전과 신뢰성을 유지하기 위해 지속적인 유지보수와 신속한 대응 전략이 얼마나 중요한지 다시 한 번 깨닫게 됩니다.

여러분은 이번 사고에 대해

## result4

In [35]:
result4=conversation({"question":prompt_template})



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway delay information through chaining and use that information to write a coherent and informative blog post using a template.
     
    
System: **Subway Delays Cause Commuter Chaos in Seoul**

On January 3rd, at approximately 5:54 PM, a train malfunction occurred between Onsu Station and Eokgok Station on the Gyeongin Line of the Seoul Metropolitan Subway, affecting over 500 passengers. These travelers were stranded inside a tunnel for about ten minutes before disembarking at Eokgok Station. The incident caused not only immediate distress among the passengers, evidenced by dramatic eyewitness videos, but also subsequent delays that impacted other commuters' schedules towards Incheon.

Although northbound trains towards Yongsan were not delayed, the event has prompted Korail to investigate the exact cause of the malfunction. This malfunction follows a previous incident in October on Line 6, wh

In [36]:
print(result4["text"])

{{서울 지하철의 예기치 못한 지연: 경인선 사고의 모든 것}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

**지연/사고 일시**: 2023년 1월 3일 오후 5:54
**지연/사고 노선**: 경인선, 온수역부터 어억곡역 사이
**지연/사고 이유**: 열차 기계 고장
**문의 사항 링크**: https://www.korail.com/customer_notice.html

1월 3일 오후 5:54분, 서울 지하철 경인선에서 온수역과 어억곡역 사이에서 열차가 갑작스럽게 멈춰 섭니다. 이로 인해 500명이 넘는 승객들이 약 10분간 터널 안에 갇히는 사태가 발생했고, 결국 어억곡역에서 승객들이 하차하여 다른 교통수단을 이용해야 했습니다. 이 사고는 다행히 큰 부상자는 없었으나, 많은 승객들에게 불편과 공포를 안겨주었고, 이후 인천 방면으로 가는 다른 열차들의 지연을 초래했습니다.

이 사건은 서울의 노후화된 지하철 인프라의 안전성과 신뢰성에 대한 우려를 다시 한번 부각시켰습니다. 과거에도 비슷한 사고가 발생한 바 있어, 정기적인 점검과 철저한 유지보수가 얼마나 중요한지를 상기시켜 줍니다. 서울시와 교통당국은 이번 사고를 계기로 철도 인프라 개선과 보다 빠른 대응 방안 마련에 총력을 기울여야 할 것입니다.

오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. 
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다. 아침 출근길, 지하철 파업으로 인해 불편을 겪고 계시다면, '지하철 온다'로 빠르고 정확한 정보를 얻어보세요. 우리 모두가 조금 더 편안한 출퇴근을 경험할 수 있도록 함께 노력합시다!

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349


In [37]:
result_final=result4["text"]

### 템플릿 평가

In [38]:
eval=["""Compare and evaluate the blog content you created with the article.
First, Extract the key information from the article. (time,date,subway line)
You need to decide whether the key information is entailed by the  CONTEXT by choosing one of the following rating: 

1. 5 point: The blog content follows logically from the information contained in the article. 

2. 1 point: The blog content is logically false from the information contained in the article. 

3. an integer score between 1 and 5 and if such integer score does not exist,  

use 1: It is not possible to determine whether the blog content is true or false without further information. 

Read the passage of information thoroughly and select the correct answer from the three answer labels. 

Read the CONTEXT thoroughly to ensure you know what the CONTEXT entails.  

Note the blog content is generated by a computer system, it can contain certain symbols, which should not be a negative factor in the evaluation.


Evaluation Form (scores ONLY):

- Consistency:
          
          """,
"""Your task is to rate the blog content on one metric.

          Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

          Evaluation Criteria:

          Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby "the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic."

          Evaluation Steps:

          1. Read the news article carefully and identify the main topic and key points.
          2. Read the blog content and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.
          3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.



          Evaluation Form (scores ONLY):

          - Coherence:
          
          """,
"""Friendliness, as a metric, measures how effectively the text of a blog content engages the reader with a friendly and inviting tone. If the tone and style of the text foster a sense of warmth and familiarity, making the reader feel welcomed and engaged, then the value of the Friendliness metric should be high; otherwise, it should be low. After reading the blog post, evaluate its tone and style against typical friendly communication, and determine the value of the Friendliness metric using the following rating scale:

          - One star: The tone is cold or unfriendly, not engaging the reader in any appreciable way.
          - Two stars: The tone has minimal warmth, slightly engaging but largely formal or distant.
          - Three stars: The tone is moderately friendly, balancing formal and informal elements to somewhat engage the reader.
          - Four stars: The tone is very friendly, actively engaging the reader with warmth and lively expressions.
          - Five stars: The tone is exceptionally friendly and lively, creating a strong sense of connection and engagement with the reader.

          This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on your assessment of how friendly and engaging the text is towards the read.

          Evaluation Template (Score Only):

          Friendliness:
          """,
"""Fluency measures the quality of individual sentences in the answer, and whether they are well-written and grammatically correct. Consider the quality of individual sentences when evaluating fluency. Given the question and answer, score the fluency of the answer between one to five stars using the following rating scale: 

          One star: the answer completely lacks fluency 

          Two stars: the answer mostly lacks fluency 

          Three stars: the answer is partially fluent 

          Four stars: the answer is mostly fluent 

          Five stars: the answer has perfect fluency 

          This rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.


          Evaluation Template (Score Only):

          Fluency:
          """,
"""Human-likeness, as a metric, evaluates how closely the style and delivery of GPT-generated text resemble that of naturally written human text. If the text is indistinguishable from human writing in terms of natural flow, vocabulary use, and contextual accuracy, then the value of the Human-likeness metric should be high; otherwise, it should be low. Given the text generated by GPT and comparing it to typical human-written text, determine the value of the Human-likeness metric using the following rating scale:

          - One star: The text is entirely machine-like, with unnatural flow and numerous contextual errors.
          - Two stars: The text is mostly machine-like, with frequent unnatural expressions and noticeable errors.
          - Three stars: The text is somewhat human-like, showing signs of natural language use but with some mechanical characteristics.
          - Four stars: The text is mostly human-like, with very few mechanical traits and generally natural language use.
          - Five stars: The text is completely human-like, indistinguishable from something a human might write.

          This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on the evaluation.
          """
]

def evaluate_blog(blog_post, evaluation_prompt, source=None):

    evaluation_result = ""

    # 정확성 이외 평가 (source 필요 없음)
    if source==None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain=prompt|llm|output_parser
        evaluation_result=chain.invoke({"input": "blog content:"+blog_post})

    # 정확성 평가 (source 필요)
    if source is not None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain = prompt | llm | output_parser
        evaluation_result=chain.invoke({"input": "blog content:"+blog_post+"article, source:"+source})
    return evaluation_result
consistency=""
coherence=""
friendliness=""
fluency=""
humanlikeness=""

정확성 평가

In [39]:
consistency=(evaluate_blog(result_final,eval[0],source_content))

Coherence 평가

In [40]:
coherence=(evaluate_blog(result_final,eval[1]))

Friendliness 평가

In [41]:
friendliness=(evaluate_blog(result_final,eval[2]))

fluency 평가

In [42]:
fluency=(evaluate_blog(result_final,eval[3]))

humanlikeness 평가

In [43]:
humanlikeness=(evaluate_blog(result_final,eval[4]))

In [ ]:
pd.DataFrame({"link":[source_link],"source":[source_content],"result":[result_final],"consistency":[consistency],"coherence":[coherence],"friendliness":[friendliness],"fluency":[fluency],"human_likeness":[humanlikeness]})

-----------

In [112]:
df=pd.concat([df,pd.DataFrame({"link":[source_link],"source":[source_content],"result":[result_final],"consistency":[consistency],"coherence":[coherence],"friendliness":[friendliness],"fluency":[fluency],"human_likeness":[humanlikeness]})])



In [118]:
print(df.iloc[1,1])


폭염 속 지하철 1호선 지연… 시민들 ‘짜증’
 강현수  승인 2023.08.01 19:01
 
전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을 호소했다.

코레일(한국철도공사)에 따르면 이날 오후 3시께부터 지하철 1호선이 10분가량 지연되고 있다.

1호선을 이용하려던 시민들은 SNS(사회관계망서비스)를 통해 "날씨가 더워서 기다리기 힘든데 지하철은 잔뜩 밀려있다", "1호선은 가뜩이나 역도 야외에 있어서 더운데 기다려야 한다" 등 반응을 보였다.

이와 관련 코레일 관계자는 "KTX(고속철도)가 폭염 영향으로 지연된 것처럼, 그 영향이 1호선에도 똑같이 적용되긴 한다"며 "3시 열차가 지연돼 뒷 열차들도 늦어지고 있는 상황"이라고 설명했다.



In [11]:
df = pd.read_json("최종지연.json")
df

,link,source,result,consistency,coherence,friendliness,fluency,human_likeness
0,https://news.kbs.co.kr/news/pc/view/view.do?nc...,출근길 서울 지하철 7호선 열차 출입문 고장…청담역·산곡역서 잠시 운행 중단 [제보...,"{{서울의 아침 출근길을 멈춘 문제의 문, 철도 문제에 대해 알아보자!}}\n{{시...",Consistency:\n5,- Coherence: 4,Friendliness:\n4,Fluency:\n5,The generated text mimics a typical blog updat...
1,https://www.news1.kr/articles/5152573%202023/0...,"""월요일부터 지각"" 지하철 1호선 지연 운행에 시민 불편…""노조 준법투쟁""\n출근 ...","{{서울 지하철 1호선, 느려진 이유는? - 파업 속출로 인한 불편을 파헤쳐보자!}...",- Consistency: 5,- Coherence: 4,Friendliness:\n4,Fluency: 5,I would rate the human-likeness of this text a...
2,https://www.joongboo.com/news/articleView.html...,폭염 속 지하철 1호선 지연… 시민들 ‘짜증’\n기자명 강현수 입력 2023.0...,"{{열기에 녹는 서울: 지하철 1호선의 시련}}\n안녕하세요, 여러분의 출퇴근 메신...",- Consistency: 5,- Coherence: 4,Friendliness:\n4,Fluency: 5,The text provided is generally well-constructe...
3,https://www.pinpointnews.co.kr/news/articleVie...,안산역 70대男 쓰러진 채 발견→4호선 한때 중단... 간편지연증명서 발급 방법·기...,{{급행 열차 대신 급한 마음: 서울 지하철 지연 증명서가 필요할 때}}\n안녕하세...,Consistency:\n5,- Coherence: 4,Friendliness:\n4,Fluency: 5,The text appears to be written in a conversati...
4,https://www.wikitree.co.kr/articles/915949,'번쩍'하더니 폭발음...어제 퇴근길 난리 난 지하철 1호선 상황 (+영상)\n20...,"{{서울 지하철, 안전은 어디에? 최근 발생한 두 차례 사고로 본 도시 철도의 문제...",- Consistency: 5,Coherence: 4,Friendliness:\n3,Fluency: 5,The blog content provided is mostly human-like...


In [23]:
print(df.iloc[1,1])

"월요일부터 지각" 지하철 1호선 지연 운행에 시민 불편…"노조 준법투쟁"
출근 시간 승객 큰 불편…지연 운행에 지각 우려
부산―수서 SRT 운행축소에 노조 24일부터 태업
(서울=뉴스1) 한병찬 기자 | 2023-08-28 08:26 송고

철도노조가 준법투쟁에 돌입한 24일 서울역 대합실에 철도노조 태업 관련 안내문이 게시돼 있다. 2023.8.24/뉴스1 © News1 김도우 기자 
전국철도노동조합(철도노조)의 준법투쟁으로 출근길 서울 지하철 1호선 이용객이 불편을 겪고 있다.

28일 뉴스1 취재를 종합하면 1호선 지하철은 철도노조의 태업으로 지연 운행하고 있다. 도심 지하철역은 출근시간대 이용객이 몰리며 큰 혼잡을 빚고 있다.

종각역으로 출근하는 20대 김모씨는 "평소보다 느리게 운행해 열차 안이 승객들로 빽빽했다"며 "철도노조 태업 때문에 운행이 지연된다는 안내방송을 하면서 급한 고객은 다른 교통수단을 이용하라는데 그러려면 몇 번이나 환승해야 해 불편하다"고 답답함을 호소했다. 

온라인에는 "1호선 연착으로 지각하게 생겼다"거나 "연착이 심해 역사 안에 발 디딜 곳이 없다" "개강 첫 날 비까지 오는데 불편하다" "퇴근길이 걱정된다"는 반응이 올라왔다. 

한국철도공사(코레일) 관계자는 "특별한 사건 사고가 있는 것은 아니기 때문에 철도노조의 준법투쟁으로 지연 운행되는 것 같다"면서 "정확한 지연 시간을 살피고 있다"고 말했다. 

철도노조는 국토교통부가 사회적 논의나 공론화 없이 내달 1일부터 부산―수서를 운행하는 SRT 고속열차의 운행을 11% 이상(일일 4100여석) 축소한 것에 반발해 지난 24일부터 준법투쟁(태업)을 하고 있다.



In [10]:
df=pd.read_csv("지연_최종5.csv")
df

,Unnamed: 0,link,source,result,consistency,coherence,friendliness,fluency,human_likeness
0,0,https://news.kbs.co.kr/news/pc/view/view.do?nc...,출근길 서울 지하철 7호선 열차 출입문 고장…청담역·산곡역서 잠시 운행 중단 [제보...,"{{서울의 아침 출근길을 멈춘 문제의 문, 철도 문제에 대해 알아보자!}}\n{{시...",Consistency:\n5,- Coherence: 4,Friendliness:\n4,Fluency:\n5,The generated text mimics a typical blog updat...
1,0,https://www.news1.kr/articles/5152573%202023/0...,"""월요일부터 지각"" 지하철 1호선 지연 운행에 시민 불편…""노조 준법투쟁""\n출근 ...","{{서울 지하철 1호선, 느려진 이유는? - 파업 속출로 인한 불편을 파헤쳐보자!}...",- Consistency: 5,- Coherence: 4,Friendliness:\n4,Fluency: 5,I would rate the human-likeness of this text a...
2,0,https://www.joongboo.com/news/articleView.html...,폭염 속 지하철 1호선 지연… 시민들 ‘짜증’\n기자명 강현수 입력 2023.0...,"{{열기에 녹는 서울: 지하철 1호선의 시련}}\n안녕하세요, 여러분의 출퇴근 메신...",- Consistency: 5,- Coherence: 4,Friendliness:\n4,Fluency: 5,The text provided is generally well-constructe...
3,0,https://www.pinpointnews.co.kr/news/articleVie...,안산역 70대男 쓰러진 채 발견→4호선 한때 중단... 간편지연증명서 발급 방법·기...,{{급행 열차 대신 급한 마음: 서울 지하철 지연 증명서가 필요할 때}}\n안녕하세...,Consistency:\n5,- Coherence: 4,Friendliness:\n4,Fluency: 5,The text appears to be written in a conversati...
4,0,https://www.wikitree.co.kr/articles/915949,'번쩍'하더니 폭발음...어제 퇴근길 난리 난 지하철 1호선 상황 (+영상)\n20...,"{{서울 지하철, 안전은 어디에? 최근 발생한 두 차례 사고로 본 도시 철도의 문제...",- Consistency: 5,Coherence: 4,Friendliness:\n3,Fluency: 5,The blog content provided is mostly human-like...


In [9]:
print(df.iloc[0,3])

{{서울의 아침 출근길을 멈춘 문제의 문, 철도 문제에 대해 알아보자!}}
{{시작하는 말}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

{{지연/사고 일시}}
2023년 10월 5일 아침 8시경

{{지연/사고 노선}}
서울 지하철 7호선

{{지연/사고 이유}}
청담역과 상곡역에서의 문제로 인한 문 장애

{{문의 사항 링크}}
https://www.seoulmetro.co.kr/kr/index.do?device=Mobile

{{뉴스기사를 이용해 블로그로 작성한 글}}
오늘 아침, 수많은 서울 시민들이 출근길에 직면한 불편함! 바로 서울 지하철 7호선에서 발생한 문 장애 사건이었습니다. 청담역과 상곡역에서 발생한 문의 문제로 인해 잠시 동안 열차 운행이 중단되었지요. 다행히도 큰 부상 소식은 없었고, 문제는 빠르게 해결되어 정상적인 운행이 재개되었습니다.

이 사고로 인해 많은 출근길 시민들이 불편을 겪었으며, 일부는 대체 교통수단을 이용해야 했습니다. 서울교통공사는 현재 정확한 사고 원인을 조사 중에 있으며, 앞으로 이런 문제가 재발하지 않도록 철저한 점검과 대책 마련에 나설 것이라고 밝혔습니다.

{{마무리 말}}
오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. 
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다. 지하철로 출근하시는 분들, 오늘처럼 갑작스런 지연이 있을 때는 조금 더 여유를 가지고 출발하시는 게 좋겠죠? 힘든 아침, 함께 이겨내요!

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349
